# Работа с видео:

1. Базовое отображение видео потока

In [ ]:
import cv2

# Инициализация 
cap = cv2.VideoCapture("video.mp4")

# Проверка, успешно ли открыто видео
if not cap.isOpened():
    print("Не удалось открыть видео")

while True:
    # Захват кадра
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр")
        break

    # ресайз
    scale = 0.5
    frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    # Отображение кадра
    cv2.imshow('video', frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()

Как сохранить результат:

In [ ]:
import cv2

# Инициализация 
cap = cv2.VideoCapture("video.mp4")

# Проверка, успешно ли открыто видео
if not cap.isOpened():
    print("Не удалось открыть видео")
    exit()

# Получение исходного FPS и размеров кадра
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Создание объекта VideoWriter для сохранения результата
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для MP4
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

while True:
    # Захват кадра 
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр")
        break

    # Ресайз
    scale = 0.5
    frame_resized = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    # Отображение кадра
    cv2.imshow('video', frame_resized)

    # Запись кадра в выходной видеофайл
    out.write(frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()

Вычисление FPS:

In [ ]:
import time
import numpy as np

class FPS_Counter:
    def __init__(self, calc_time_perion_N_frames: int) -> None:
        """Счетчик FPS по ограниченным участкам видео (скользящему окну).

        Args:
            calc_time_perion_N_frames (int): количество фреймов окна подсчета статистики.
        """
        self.time_buffer = []
        self.calc_time_perion_N_frames = calc_time_perion_N_frames

    def calc_FPS(self) -> float:
        """Производит рассчет FPS по нескольким кадрам видео.

        Returns:
            float: значение FPS.
        """
        time_buffer_is_full = len(self.time_buffer) == self.calc_time_perion_N_frames
        t = time.time()
        self.time_buffer.append(t)

        if time_buffer_is_full:
            self.time_buffer.pop(0)
            fps = len(self.time_buffer) / (self.time_buffer[-1] - self.time_buffer[0])
            return np.round(fps, 2)
        else:
            return 0.0


def calc_and_show_fps(frame, fps_counter):
    """
    Вычисляет и отображает FPS на кадре.

    Args:
        frame (numpy.ndarray): Текущий кадр изображения.
        fps_counter (FPS_Counter): Объект класса FPS_Counter для вычисления FPS.

    Returns:
        numpy.ndarray: Кадр с отображенным значением FPS.
    """
    fps_real = fps_counter.calc_FPS()
    text = f"FPS: {fps_real:.1f}"

    # Параметры для шрифтов:
    fontFace = 1
    fontScale = 1.3
    thickness = 1
    
    (label_width, label_height), _ = cv2.getTextSize(
        text,
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness,
    )
    frame = cv2.rectangle(frame, (0, 0), (10 + label_width, 15 + label_height), (0, 0, 0), -1)
    frame = cv2.putText(
        img=frame,
        text=text,
        org=(5, 20),
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness,
        color=(255, 255, 255),
    )
    return frame

In [ ]:
fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Инициализация
cap = cv2.VideoCapture("video.mp4")

# Проверка, успешно ли открыто видео
if not cap.isOpened():
    print("Не удалось открыть видео")
    exit()

# Получение исходного FPS и размеров кадра
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Создание объекта VideoWriter для сохранения результата
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для MP4
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

while True:
    # Захват кадра 
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр")
        break

    # Ресайз
    scale = 0.5
    frame_resized = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)
    frame_resized = calc_and_show_fps(frame_resized, fps_counter)

    # Отображение кадра
    cv2.imshow('video', frame_resized)

    # Запись кадра в выходной видеофайл
    out.write(frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()
    

## Инференс нейронной сети YOLO:

In [1]:
!pip install patched-yolo-infer

In [2]:
from ultralytics import YOLO

from patched_yolo_infer import (
    visualize_results_usual_yolo_inference,
)

Получим один кадр для подстройки:

In [3]:
# Инициализация 
cap = cv2.VideoCapture("video.mp4")

# Проверка, успешно ли открыта веб-камера
if not cap.isOpened():
    print("Не удалось открыть веб-камеру")

# Захват кадра 
ret, frame = cap.read()

NameError: name 'cv2' is not defined

In [ ]:
# Define the parameters
imgsz = 640
conf = 0.25
iou = 0.7

# Load the YOLOv8 model
model = YOLO("yolov8m-seg.pt") 

visualize_results_usual_yolo_inference(
    frame,
    model,
    imgsz,
    conf,
    iou,
    segment=True,
    delta_colors=3,
    thickness=8,
    font_scale=2.0,
    show_boxes=False,
    fill_mask=True,
    alpha=0.2,
    show_confidences=True,
)

Инстанс сегментация:

In [ ]:
# Define the parameters
imgsz = 640
conf = 0.25
iou = 0.7

# Load the YOLOv8 model
model = YOLO("yolov8m-seg.pt")  

fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Инициализация 
cap = cv2.VideoCapture("video.mp4")

# Проверка, успешно ли открыто видео
if not cap.isOpened():
    print("Не удалось открыть видео")
    exit()

# Получение исходного FPS и размеров кадра
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Создание объекта VideoWriter для сохранения результата
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для MP4
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

while True:
    # Захват кадра 
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр")
        break

    frame = visualize_results_usual_yolo_inference(
        frame,
        model,
        imgsz,
        conf,
        iou,
        segment=True,
        delta_colors=3,
        thickness=8,
        font_scale=2.0,
        show_boxes=False,
        fill_mask=True,
        alpha=0.35,
        show_confidences=True,
        return_image_array=True
    )

    # Ресайз
    scale = 0.5
    frame_resized = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)
    frame_resized = calc_and_show_fps(frame_resized, fps_counter)

    # Отображение кадра
    cv2.imshow('video', frame_resized)

    # Запись кадра в выходной видеофайл
    out.write(frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()
    

Детекция:

In [ ]:
# Define the parameters
imgsz = 640
conf = 0.25
iou = 0.7

# Load the YOLOv8 model
model = YOLO("yolov8m.pt") 

fps_counter = FPS_Counter(calc_time_perion_N_frames=10) # Определяем размер усреднения

# Инициализация 
cap = cv2.VideoCapture("video.mp4")

# Проверка, успешно ли открыто видео
if not cap.isOpened():
    print("Не удалось открыть видео")
    exit()

# Получение исходного FPS и размеров кадра
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Создание объекта VideoWriter для сохранения результата
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Кодек для MP4
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

while True:
    # Захват кадра 
    ret, frame = cap.read()

    # Проверка, успешно ли захвачен кадр
    if not ret:
        print("Не удалось получить кадр")
        break

    frame = visualize_results_usual_yolo_inference(
        frame,
        model,
        imgsz,
        conf,
        iou,
        segment=False,
        delta_colors=3,
        thickness=8,
        font_scale=2.0,
        show_boxes=True,
        show_confidences=True,
        return_image_array=True
    )

    # Ресайз
    scale = 0.5
    frame_resized = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)
    frame_resized = calc_and_show_fps(frame_resized, fps_counter)

    # Отображение кадра
    cv2.imshow('video', frame_resized)

    # Запись кадра в выходной видеофайл
    out.write(frame)

    # Выход из цикла по нажатию клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
out.release()
cv2.destroyAllWindows()
    